In [1]:
%%time

import os
import re
import string
import pprint

import pandas as pd
import numpy as np

import gensim

Wall time: 16.1 s


In [2]:
%%time

# change working directory

print('Before chdir:', os.getcwd())

os.chdir('C:\\Users\\ravigv\\Documents\\cosine_sim\\cosimV3') 
         
print('After chdir:', os.getcwd())

Before chdir: C:\Users\ravigv
After chdir: C:\Users\ravigv\Documents\cosine_sim\cosimV3
Wall time: 1 ms


In [3]:
%%time

df=pd.read_excel("wb_causecode_asgrp.xlsx")
print("before", df.shape)
df = df.dropna()
print("After", df.shape)

df['combi_feat'] = df['summary']+' '+df['causecode_lv1']+' '+df['causecode_lv2']+' '+df['causecode_lv3']
df['a_cclvl123'] =   df['causecode_lv1']+' '+df['causecode_lv2']+' '+df['causecode_lv3']
df.head()


before (73647, 8)
After (73493, 8)
Wall time: 7.61 s


,account,app_name,assigned_group,causecode_lv1,causecode_lv2,causecode_lv3,incident_id,summary,combi_feat,a_cclvl123
1,wb,SAP Accounts Payable/Workflow,WBEI.MKTMGT.Siebel North America,Delete,Data Deletion,As per user requirement,RTASK0086491,"Promo1169410 ACB#236740 Deal Status ""Payment i...","Promo1169410 ACB#236740 Deal Status ""Payment i...",Delete Data Deletion As per user requirement
3,wb,SAP Netweaver Business Warehouse,WBEI.BW - Data Warehouse,Access,Approval,Access/SAP Issue/Unable to access environment,RTASK0246906,FRB Reconciliation need to be perform on 06/04...,FRB Reconciliation need to be perform on 06/04...,Access Approval Access/SAP Issue/Unable to ac...
4,wb,System,WBEI.WBCAS Adobe LiveCycle,Access,Approval,Access/SAP Issue/Unable to access environment,RTASK0286950,!! URGENT-- Forward eform 11828 to another app...,!! URGENT-- Forward eform 11828 to another app...,Access Approval Access/SAP Issue/Unable to ac...
5,wb,SAP BUSINESS OBJECTS,WBEI.BO.Admin,Request,Enquiry,Information,RTASK0300853,Report Migration From DEV to QA,Report Migration From DEV to QA Request Enquir...,Request Enquiry Information
6,wb,EI (Electronic Invoice),WBEI.SC.EI,Access,Request,Unlock,RTASK0341560,Disable EI Batch Jobs,Disable EI Batch Jobs Access Request Unlock,Access Request Unlock


In [4]:
df = df[['incident_id', 'summary','combi_feat', 'a_cclvl123']]
df.head()

,incident_id,summary,combi_feat,a_cclvl123
1,RTASK0086491,"Promo1169410 ACB#236740 Deal Status ""Payment i...","Promo1169410 ACB#236740 Deal Status ""Payment i...",Delete Data Deletion As per user requirement
3,RTASK0246906,FRB Reconciliation need to be perform on 06/04...,FRB Reconciliation need to be perform on 06/04...,Access Approval Access/SAP Issue/Unable to ac...
4,RTASK0286950,!! URGENT-- Forward eform 11828 to another app...,!! URGENT-- Forward eform 11828 to another app...,Access Approval Access/SAP Issue/Unable to ac...
5,RTASK0300853,Report Migration From DEV to QA,Report Migration From DEV to QA Request Enquir...,Request Enquiry Information
6,RTASK0341560,Disable EI Batch Jobs,Disable EI Batch Jobs Access Request Unlock,Access Request Unlock


In [5]:
# # Spliting data into train and tst set

# # Train set with 90% values of original dataframe
# train = df.sample(frac = 0.99)
# print("train", train.shape)
  
# # Creating dataframe with rest of the 10% values
# test = df.drop(train.index)

# print("test", test.shape)

In [6]:
# train1 = df.sample(n=73483, replace=False)

# print("train1", train1.shape)

# test1 = df.drop(train1.index)
# test1 = test1.reset_index(drop = True)
# print("test1", test1.shape)

# train1 = train1.reset_index(drop = True)



In [7]:
test1 = df.head(100)

train1 = df.drop(test1.index)
train1 = train1.reset_index(drop = True)

test1 = test1.reset_index(drop = True)


print("train1", train1.shape)
print("test1", test1.shape)

train1 (73393, 4)
test1 (100, 4)


In [8]:
# train1.to_excel("out_cosim/wb_train1_V1.xlsx")

# test1.to_excel("out_cosim/wb_test1_10_V1.xlsx")

In [9]:
train1.head()

,incident_id,summary,combi_feat,a_cclvl123
0,RTASK1149837,ZP doctypes not being cleared properly.,ZP doctypes not being cleared properly. Reque...,Request Enquiry Information
1,RTASK1149994,Estimate:- Two issues with vendor updates in PSL,Estimate:- Two issues with vendor updates in ...,Request Estimate Enhancement
2,RTASK1151780,CRM: Estimate effort to Turn RIM off for Carre...,CRM: Estimate effort to Turn RIM off for Carre...,Data Missing Field missing in documents
3,RTASK1152184,[URGENT] Clear Code SAP,[URGENT] Clear Code SAP Request Enquiry Inform...,Request Enquiry Information
4,RTASK1152397,Unable to Reject the Item,Unable to Reject the Item Application Code Bug...,Application Code Bug Fix


In [10]:
test1

,incident_id,summary,combi_feat,a_cclvl123
0,RTASK0086491,"Promo1169410 ACB#236740 Deal Status ""Payment i...","Promo1169410 ACB#236740 Deal Status ""Payment i...",Delete Data Deletion As per user requirement
1,RTASK0246906,FRB Reconciliation need to be perform on 06/04...,FRB Reconciliation need to be perform on 06/04...,Access Approval Access/SAP Issue/Unable to ac...
2,RTASK0286950,!! URGENT-- Forward eform 11828 to another app...,!! URGENT-- Forward eform 11828 to another app...,Access Approval Access/SAP Issue/Unable to ac...
3,RTASK0300853,Report Migration From DEV to QA,Report Migration From DEV to QA Request Enquir...,Request Enquiry Information
4,RTASK0341560,Disable EI Batch Jobs,Disable EI Batch Jobs Access Request Unlock,Access Request Unlock
...,...,...,...,...
95,RTASK1146573,Ticket for the team,Ticket for the team Data Mismatch Records miss...,Data Mismatch Records missing in table
96,RTASK1147331,Complete estimate,Complete estimate Request Enquiry Information,Request Enquiry Information
97,RTASK1147829,Complete request - SAP system open access process,Complete request - SAP system open access proc...,Access SAP Issue Unable to access environment
98,RTASK1148277,Monitoring Services - Great Plains AR,Monitoring Services - Great Plains AR Access R...,Access Request Password Reset


In [11]:
%%time

# Function to keep required columns in a dataframe

def keepcols(df):   
    return df[['incident_id','causecode_lv1', 'causecode_lv2', 'causecode_lv3', 'combi_feat']]


# Checking and imputing missing values

def check_replace_missing(df, na_fill):
#     print("Before imputing missing values", df1.isnull().values.any())
    df = df.fillna(na_fill)
#     print("After imputing missing values", df.isnull().values.any())
    return df



# Preprocess text 
import re
from gensim.parsing.preprocessing import remove_stopwords

def clean_sentence(sentence, stopwords=False):
    
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    #lower case 
    sentence = sentence.lower()
    # square brackets
    sentence = re.sub('\[.*?\]', '', sentence)  
    # hyperlinks
    sentence = re.sub('https?://\S+|www\.\S+', '', sentence)
    sentence = re.sub('<.*?>+', '', sentence)
    # remove punctuation
    sentence = re.sub('[%s]' % re.escape(string.punctuation), '', sentence)
    sentence = re.sub('\n', '', sentence)
    # remove year
    sentence = re.sub(r"\b(19[40][0-9]|20[0-1][0-9]|2020)\b",'',sentence)
    # remove month names
    mp = r"(\b\d{1,2}\D{0,3})?\b(?:Jan|jan(?:uary)?|Feb|feb(?:ruary)?|Mar|mar(?:ch)?|Apr|mar(?:il)?|May|may|Jun|jun(?:e)?|Jul|jul(?:y)?|Aug|aug(?:ust)?|Sep|sep(?:tember)?|Oct|oct(?:ober)?|(Nov|nov|Dec|dec)(?:ember)?)\D?(\d{1,2}(st|nd|rd|th)?)?(([,.\-\/])\D?)?((19[7-9]\d|20\d{2})|\d{2})*"
    sentence = re.sub(mp, '', sentence)
     
    # remove words containing numbers
    sentence = re.sub('\w*\d\w*', '', sentence)
    # remove numbers 
    sentence = re.sub(r'[0-9]+','',sentence)
    
    if stopwords:
        sentence = remove_stopwords(sentence)
    
    return sentence

# Get clean sentences with numpy
def get_cleaned_sents(df, col_name, stopwords=False):
   # find the index no 
    index_no = df.columns.get_loc(col_name)
    # Convert to numpy array
    df_np = df.to_numpy()
    
    rows = [clean_sentence(row[index_no]) for row in df_np]
    return rows


# Getting Phrase Embeddings from Word Embeddings
def getWordVec(word,model):
        samp=model['users']
        vec=[0]*len(samp);
        try:
                vec=model[word]
        except:
                vec=[0]*len(samp)
        return (vec)
    

def getPhraseEmbedding(phrase,embeddingmodel):
                       
        samp=getWordVec('users', embeddingmodel)
        vec=np.array([0]*len(samp));
        den=0;
        for word in phrase.split():
            #print(word)
            den=den+1;
            vec=vec+np.array(getWordVec(word,embeddingmodel))
        #vec=vec/den;
        #return (vec.tolist());
        return vec.reshape(1, -1)
 
    
# Get cosine similarity    
import sklearn
from sklearn.metrics.pairwise import cosine_similarity


def get_cosim(query_embedding,sentence_embeddings,sentences,topn, DF):
    count = []
    simscore = []
    result = []
    inc_id = []
    
# sentences=cleaned_sentences # for inedexing at results.append
    for index,faq_embedding in enumerate(sentence_embeddings):
        try:
            sim=cosine_similarity(faq_embedding,query_embedding)[0][0]

            count.append(index)
            simscore.append(sim)
            result.append(sentences[index])
            inc_id.append(DF.loc[index, 'incident_id'])
        
        except Exception as e:
#             print(e)
            pass



    dfScore = pd.DataFrame(list(zip(count, simscore, result, inc_id)),
              columns=['id','simscore', 'result', 'incident_id'])


    dfsort = dfScore.sort_values('simscore', ascending = False)
    
    return dfsort[:topn]



# Case 1: Get top similarities a function wrapping all above - without parallelization
import pathlib

def get_topsim(df, query, model_name):
    
    # Load the model
    model = pathlib.Path(f"./{model_name}.txt")
    print(model)
    if model.exists():
        model_name = gensim.models.KeyedVectors.load_word2vec_format(f"./{model_name}.txt", binary=False)
#         print ("model exists and loaded successfully")
    else:
        print ("model does not exist")
    
#     df = keepcols(df) 
#     print("Read and subsetted data")
#     df = check_replace_missing(df, '')
#     print("Finished imputing missing values")
    cleaned_sentences = get_cleaned_sents(df, 'combi_feat')
    sentences = cleaned_sentences
#     print("Finished cleaning sentences")
    
    # Sentence wmbedings
    sent_embeddings=[]
    for sent in sentences:
        sent_embeddings.append(getPhraseEmbedding(sent,model_name))
        
#     print("Finished creating sentence embedings")

    # Get question embeding
    query_clean=clean_sentence(query,stopwords=False);
    question_embedding=getPhraseEmbedding(query_clean,model_name)
#     print("Finished creating question embedings")

    # Get top5 cosine similarity for query and sentences
    dff = get_cosim(question_embedding,sent_embeddings,sentences, 1, df)
#     print("Finished cosim process")
    return dff

  


Wall time: 1.38 s


#### Case1: Without dask parallelization

In [12]:
%%time

query= "FRB Reconciliation need to be perform on 06/04/2015"
query=clean_sentence(query,stopwords=False);
# query= "user is unable to login to sap server"
get_topsim(train1, query, 'wb_word2vec_model')

wb_word2vec_model.txt
Wall time: 28.4 s


,id,simscore,result,incident_id
68534,68534,0.808502,clscritical unable to perform usd fca costing...,SINC2416401


In [13]:
query

'frb reconciliation need to be perform on '

#### Case2: Many to many similarity search

In [14]:
%%time
df_final = pd.DataFrame()

# queries = []
for i in range(len(test1)):
    try:
    #     print(test.at[i,'summary'])
        query = test1.at[i,'combi_feat']
#         queries.extend([query for i in range(5)])
        dfsim = get_topsim(train1, query, 'wb_word2vec_model')
        frames = [df_final, dfsim]
        df_final = pd.concat(frames)
    except Exception as e:
                
#             print(e)
        pass

wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2vec_model.txt
wb_word2ve

In [15]:
print(df_final.shape)



df_final



(100, 4)


,id,simscore,result,incident_id
47971,47971,0.847134,urgent deal the errors on the deal were remov...,SINC0957305
400,400,0.905697,frb recon on access approval accesssap issue...,RTASK1186970
30713,30713,0.891805,configure bethan fallon with £ gbp po approval...,RTASK1750898
25049,25049,0.893574,migration of report from dev to general reque...,RTASK1632445
59144,59144,0.876521,foxpro billing unable to locate batch prepro...,SINC1724197
...,...,...,...,...
57687,57687,0.918049,imabalances created on the job for station d...,SINC1671116
17644,17644,1.000000,complete estimate request enquiry information,RTASK1496080
22929,22929,0.993105,complete request sap system open access proce...,RTASK1595059
5844,5844,1.000000,monitoring services great plains ar access re...,RTASK1271306


In [16]:
print(df_final.shape)

(100, 4)


In [17]:
df_final.to_excel("out_cosim/df_final_V2_6.xlsx")

#### Merge results df with train df by incident id

In [18]:
dfm = pd.merge(df_final[['result', 'simscore','incident_id' ]],
                           train1[['incident_id', 'a_cclvl123', 'combi_feat']], 
                           left_on='incident_id', 
                           right_on='incident_id', 
                           how='left')

In [19]:
print(dfm.shape)

dfm

(100, 5)


,result,simscore,incident_id,a_cclvl123,combi_feat
0,urgent deal the errors on the deal were remov...,0.847134,SINC0957305,Data Data flow Data flow failure,URGENT Deal 3009126 The errors on the deal wer...
1,frb recon on access approval accesssap issue...,0.905697,RTASK1186970,Access Approval Access/SAP Issue/Unable to ac...,FRB Recon on 08/14 Access Approval Access/SAP...
2,configure bethan fallon with £ gbp po approval...,0.891805,RTASK1750898,Access Request Eform,Configure Bethan Fallon with £250K GBP PO Appr...
3,migration of report from dev to general reque...,0.893574,RTASK1632445,General Request Estimate,Migration of report from DEV to QA7 General Re...
4,foxpro billing unable to locate batch prepro...,0.876521,SINC1724197,Access Request Password Reset,[URGENT] Foxpro Billing unable to locate batc...
...,...,...,...,...,...
95,imabalances created on the job for station d...,0.918049,SINC1671116,Data Mismatch Records missing in table,Imabalances created on the 2hrly job for KYOU-...
96,complete estimate request enquiry information,1.000000,RTASK1496080,Request Enquiry Information,Complete estimate Request Enquiry Information
97,complete request sap system open access proce...,0.993105,RTASK1595059,Access Issue Unable to access environment,Complete request - SAP system open access proc...
98,monitoring services great plains ar access re...,1.000000,RTASK1271306,Access Request Password Reset,Monitoring Services - Great Plains AR Access R...


In [20]:
queries = []
p_cclvl123 = []

for i in range(len(test1)):
    query1 = test1.at[i,'combi_feat']
    clvl1123 = test1.at[i,'a_cclvl123']
        
    queries.extend([query1 for i in range(1)])
    p_cclvl123.extend([clvl1123 for i in range(1)])
    

In [21]:
len(queries)

100

In [22]:
dfm['quriesz'] = queries
dfm['p_cclvl123'] = p_cclvl123

dfm

,result,simscore,incident_id,a_cclvl123,combi_feat,quriesz,p_cclvl123
0,urgent deal the errors on the deal were remov...,0.847134,SINC0957305,Data Data flow Data flow failure,URGENT Deal 3009126 The errors on the deal wer...,"Promo1169410 ACB#236740 Deal Status ""Payment i...",Delete Data Deletion As per user requirement
1,frb recon on access approval accesssap issue...,0.905697,RTASK1186970,Access Approval Access/SAP Issue/Unable to ac...,FRB Recon on 08/14 Access Approval Access/SAP...,FRB Reconciliation need to be perform on 06/04...,Access Approval Access/SAP Issue/Unable to ac...
2,configure bethan fallon with £ gbp po approval...,0.891805,RTASK1750898,Access Request Eform,Configure Bethan Fallon with £250K GBP PO Appr...,!! URGENT-- Forward eform 11828 to another app...,Access Approval Access/SAP Issue/Unable to ac...
3,migration of report from dev to general reque...,0.893574,RTASK1632445,General Request Estimate,Migration of report from DEV to QA7 General Re...,Report Migration From DEV to QA Request Enquir...,Request Enquiry Information
4,foxpro billing unable to locate batch prepro...,0.876521,SINC1724197,Access Request Password Reset,[URGENT] Foxpro Billing unable to locate batc...,Disable EI Batch Jobs Access Request Unlock,Access Request Unlock
...,...,...,...,...,...,...,...
95,imabalances created on the job for station d...,0.918049,SINC1671116,Data Mismatch Records missing in table,Imabalances created on the 2hrly job for KYOU-...,Ticket for the team Data Mismatch Records miss...,Data Mismatch Records missing in table
96,complete estimate request enquiry information,1.000000,RTASK1496080,Request Enquiry Information,Complete estimate Request Enquiry Information,Complete estimate Request Enquiry Information,Request Enquiry Information
97,complete request sap system open access proce...,0.993105,RTASK1595059,Access Issue Unable to access environment,Complete request - SAP system open access proc...,Complete request - SAP system open access proc...,Access SAP Issue Unable to access environment
98,monitoring services great plains ar access re...,1.000000,RTASK1271306,Access Request Password Reset,Monitoring Services - Great Plains AR Access R...,Monitoring Services - Great Plains AR Access R...,Access Request Password Reset


In [23]:
dfm.to_excel("out_cosim/wb_train_vs_wb_test10_V2_6.xlsx")

In [24]:
# for i in range(len(test1)):
#     print(i)
#     print(test1.at[i,'combi_feat'])